## 1. urllib 활용

In [31]:
import requests as r
import urllib.request as ur
import os
# import pandas as pd
import time
import pprint


In [3]:
# 해당 디렉토리가 없으면 만드는 함수
def createFolder(dir_name):
    try:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    except OSError:
        print ('Error: Creating directory. ' +  dir_name)

In [4]:
# 쿼리를 요청할 검색어 리스트 만들거나 불러오기
name_list = ['공모전']

In [47]:
key1 = " " # Your bing API key
endpoint = "https://api.bing.microsoft.com/" 
url  = f"{endpoint}/v7.0/images/search"

In [53]:
headers = {
            "Ocp-Apim-Subscription-Key": key1
        }
param = {
            "q": "공모전",
            "license": "public",
            "imageType": "photo", 
            "safeSearch": "strict"
        }

In [54]:
response = r.get(url, headers=headers, params=param)
response.raise_for_status()
pprint.pprint(response.json())

HTTPError: 404 Client Error: Resource Not Found for url: https://api.bing.microsoft.com/?q=ss&license=public&safeSearch=strict

In [30]:
def make_img_list(lst, apikey):
    # 시작 시간 저장
    start = time.time()
    # 에러 로그 저장을 위한 리스트 만들기
    error_log = []
    # 주어진 리스트에 있는 모든 이름 검색
    for item in lst:

        # 폴더 만들고 경로 이동하기
        base_dir = "/Users/jincheol/Desktop/종합 설계/Dataset/bing search"
        if not os.path.exists(base_dir):
            os.makedirs(base_dir)  # 경로가 없으면 생성

        os.chdir(base_dir)
        createFolder(item)
        os.chdir(f"./{item}")

        url = "https://api.bing.microsoft.com/v7.0/images/search"
        #url = "https://api.bing.microsoft.com/"
        headers = {
            "Ocp-Apim-Subscription-Key": apikey
        }
        param = {
            "q": item,
            "license": "public",
            "count": 500,
            "imageType": "Photo",
            "minHeight": 256,
            "safeSearch": "strict",
            "color": "ColorOnly"
        }
        s = r.Session()
        response = s.get(url, headers=headers, params=param)
        
        # 응답 상태 코드 확인
        if response.status_code != 200:
            print(f"Error: API request failed with status code {response.status_code}")
            continue
        
        test = response.json()
        
        # API 응답 디버깅 출력
        print("API 응답:", test)
        
        # 검색결과 링크에서 이미지 저장하기
        if "value" in test:
            for i in range(0, len(test.get("value"))):
                print(item, '-', i)
                print(test.get('value')[i].get('contentUrl'))
                file_name = f"{item}_{i}.{test.get('value')[i].get('encodingFormat')}"
                try:
                    ur.urlretrieve(test.get('value')[i].get('contentUrl'), file_name)
                except Exception as e:
                    print(e)
                    error_log.append(f"{file_name}: {e}")
                    pass
        else:
            print(f"No images found for {item}")
        
        os.chdir("../..")
    time_taken = time.time() - start
    return time_taken, error_log

In [25]:
make_img_list(name_list, key1)

Error: API request failed with status code 401
Error: API request failed with status code 401
Error: API request failed with status code 401


(0.6941008567810059, [])

In [58]:
from azure.cognitiveservices.search.imagesearch import ImageSearchClient
from msrest.authentication import CognitiveServicesCredentials

In [66]:
subscription_key = "bec27c152ce34c95a902e517abe1722b"
subscription_endpoint = "https://api.bing.microsoft.com/"
search_term = "공모전" 

In [67]:
client = ImageSearchClient(endpoint=subscription_endpoint, credentials=CognitiveServicesCredentials(subscription_key))

In [68]:
image_results = client.images.search(query=search_term)
if image_results.value:
    first_image_result = image_results.value[0]
    print("Total number of images returned: {}".format(len(image_results.value)))
    print("First image thumbnail url: {}".format(
        first_image_result.thumbnail_url))
    print("First image content url: {}".format(first_image_result.content_url))
else:
    print("No image results returned!")

ErrorResponseException: Operation returned an invalid status code 'Resource Not Found'